In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('NHK_program_text_data.csv')

In [3]:
df.head()

,info,genre_name_detailed
0,北アルプス「北アルプスの名峰」。北アルプスに位置する百名山をはじめとする名峰の数々。,自然・動物・環境
1,みんなのうた「わたしの紙風船」「天下無敵のゴーヤーマン★」。「わたしの紙風船」うた：紙ふうせ...,童謡・キッズ
2,みんなのうた「わたしの紙風船」「天下無敵のゴーヤーマン★」。「わたしの紙風船」うた：紙ふうせ...,国内アニメ
3,みんなのうた「わたしの紙風船」「天下無敵のゴーヤーマン★」。「わたしの紙風船」うた：紙ふうせ...,音楽バラエティ
4,キャラとおたまじゃくし島（３４）「音楽堂のガッキアニマル の巻」。まっすぐで愛にあふれた少女...,国内アニメ


In [4]:
df['genre_name_detailed'].unique()

array(['自然・動物・環境', '童謡・キッズ', '国内アニメ', '音楽バラエティ', '番組紹介・お知らせ', '定時・総合',
       '特集・ドキュメント', 'スポーツニュース', 'ローカル・地域', '交通', '国内ドラマ', '暮らし・住まい',
       '健康・医療', '社会・時事', 'その他', '天気', '幼児・小学生', '旅・釣り・アウトドア', '歴史・紀行',
       '旅バラエティ', 'ドキュメンタリー全般', 'グルメ・料理', '料理バラエティ', 'カルチャー・伝統文化',
       'トークバラエティ', 'お笑い・コメディ', '解説', '討論・会談', '国内ロック・ポップス', '海外ロック・ポップス',
       '海外・国際', '相撲・格闘技', '中学生・高校生', '会話・語学', '音楽・美術・工芸', '手話', '高齢者',
       '障害者', '生涯教育・資格', '民族音楽・ワールドミュージック', '文学・文芸', '園芸・ペット・手芸',
       'コンピュータ・TVゲーム', 'クラシック・オペラ', '歌舞伎・古典', '民謡・邦楽', '社会福祉', '宇宙・科学・医学',
       '大学生・受験', '落語・演芸', 'クイズ', '海外アニメ', 'ゴルフ', 'オリンピック・国際大会',
       '芸能・ワイドショー', 'スポーツ', '野球', 'ライブ・コンサート', '洋画', '時代劇', '歌謡曲・演歌',
       '政治・国会', '経済・市況', '報道特番', '教育問題', '囲碁・将棋', '海外ドラマ', 'ファッション',
       'インタビュー・討論', 'マリン・ウィンタースポーツ', 'その他の球技', 'ゲーム', '邦画', 'ジャズ・フュージョン',
       'マラソン・陸上・水泳', '特撮', 'サッカー', 'ボランティア', 'ダンス・バレエ', 'イベント', '現代劇・新劇',
       'ミュージカル', 'カラオケ・のと゛自慢'], dtype=object)

## '自然・動物・環境'

In [5]:
nature = df[df.genre_name_detailed == '自然・動物・環境']
for idx, row in enumerate(nature['info']):
    with open(f'data/texts/nature/nature_{idx}.txt','w',encoding='utf-8') as text:
        text.write(row)

## '中学生・高校生'

In [6]:
students = df[df.genre_name_detailed == '中学生・高校生']
for idx, row in enumerate(students['info']):
    with open(f'data/texts/students/students_{idx}.txt','w',encoding='utf-8') as text:
        text.write(row)

## '特撮'

In [7]:
food = df[df.genre_name_detailed == 'グルメ・料理']
for idx, row in enumerate(food['info']):
    with open(f'data/texts/food/food_{idx}.txt','w',encoding='utf-8') as text:
        text.write(row)

## '健康・医療'

In [8]:
health = df[df.genre_name_detailed == '健康・医療']
for idx, row in enumerate(health['info']):
    with open(f'data/texts/health/health_{idx}.txt','w',encoding='utf-8') as text:
        text.write(row)

## Create CSV paths

In [9]:
root = 'gs://nhk_text_classification/'

In [10]:
import glob

In [11]:
text_paths = glob.glob('data/texts/*/*')

In [12]:
files = [root + x.split('\\')[-2] +'/'+ x.split('\\')[-1] for x in text_paths]

In [13]:
txt_df = pd.DataFrame()
txt_df['label'] = ['UNASSIGNED'] * len(text_paths) 
txt_df['file_path'] = files
txt_df['target'] = [x.split('\\')[-2] for x in text_paths]

In [14]:
txt_df

,label,file_path,target
0,UNASSIGNED,gs://nhk_text_classification/food/food_0.txt,food
1,UNASSIGNED,gs://nhk_text_classification/food/food_1.txt,food
2,UNASSIGNED,gs://nhk_text_classification/food/food_10.txt,food
3,UNASSIGNED,gs://nhk_text_classification/food/food_100.txt,food
4,UNASSIGNED,gs://nhk_text_classification/food/food_101.txt,food
...,...,...,...
1804,UNASSIGNED,gs://nhk_text_classification/students/students...,students
1805,UNASSIGNED,gs://nhk_text_classification/students/students...,students
1806,UNASSIGNED,gs://nhk_text_classification/students/students...,students
1807,UNASSIGNED,gs://nhk_text_classification/students/students...,students


In [15]:
txt_df.to_csv('txt_labels.csv',index=0,header=0)

## Call to the Model API

In [16]:
from google.cloud import automl
import os

#For google authentication purpose.
#Update full path of service account credential file (JSON file)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "secert/AutoML_NHKProgramClassification_ServiceKey.json"

In [17]:
sample_text = 'いじめをノックアウト「ＳＮＳに友だちの悪口…見つけたら　どうする？」。\
                ある日、友だちのＳＮＳを見たら、プロフィール欄にクラスメイトの悪口が…。\
                そこから、いじめを生まないために、周りの人は何ができる？高橋みなみさんの解決法は？。\
                「女王様気どりもいい加減にして！」本来は自己紹介などを書くＳＮＳのプロフィール欄に、\
                あえて名指しせずに悪口を書くいじめが、子どもたちの間で起きていると言う。大阪市立田島中学校の生徒会では、\
                あるエピソードをもとに「どうすれば止められるのか」を討論。書き込みに反応しない、先生に言う、そして肩を組む！？\
                さまざまなアイデアを出し合って解決の糸口を探す。高橋みなみさんも解決法を発案！カギは「傍観者の仲間」？'

model_id = 'TCN2217038753573109760'
project_id = 'alpine-sentry-349911'
compute_region = 'us-central1'

In [21]:
prediction_client = automl.PredictionServiceClient()

# Get the full path of the model.
model_full_id = automl.AutoMlClient.model_path(project=project_id, 
                                               location=compute_region, 
                                               model=model_id)

text_snippet = automl.TextSnippet(content=sample_text, 
                                  mime_type="text/plain")

payload = automl.ExamplePayload(text_snippet=text_snippet)

response = prediction_client.predict(name=model_full_id, payload=payload)

print(response)

payload {
  annotation_spec_id: "6567403293671161856"
  classification {
    score: 0.5809033
  }
  display_name: "health"
}
payload {
  annotation_spec_id: "1955717275243773952"
  classification {
    score: 0.26224184
  }
  display_name: "students"
}
payload {
  annotation_spec_id: "7720324798278008832"
  classification {
    score: 0.1523304
  }
  display_name: "nature"
}
payload {
  annotation_spec_id: "4261560284457467904"
  classification {
    score: 0.0045245085
  }
  display_name: "food"
}



In [26]:
for annotation_payload in response.payload:

    print(f"Predicted class | Score\n{annotation_payload.display_name} | {annotation_payload.classification.score}\n")


Predicted class | Score
health | 0.5809032917022705

Predicted class | Score
students | 0.26224184036254883

Predicted class | Score
nature | 0.1523303985595703

Predicted class | Score
food | 0.004524508491158485

